# Darcy equation: exercise 1

In this tutorial we present how to solve a Darcy equation with [PyGeoN](https://github.com/compgeo-mox/pygeon).  The unknowns are the velocity $q$ and the pressure $p$.

Let $\Omega=(0,1)^2$ with boundary $\partial \Omega$ and outward unit normal ${\nu}$. Given 
$k=I$ the matrix permeability and $f = 1$ the source term, we want to solve the following problem: find $({q}, p)$ such that
$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
{q} + \nabla p = 0\\
\nabla \cdot {q} = f
\end{array}
&\text{in } \Omega
\end{array}
\right.
$$
with boundary conditions:
$$ p = 0 \text{ on } \partial \Omega$$

We present *step-by-step* how to create the grid, declare the problem data, and finally solve the problem.

First we import some of the standard modules, like `numpy` and `scipy.sparse`. Since PyGeoN is based on [PorePy](https://github.com/pmgbergen/porepy) we import both modules.

In [1]:
import numpy as np
import scipy.sparse as sps

import porepy as pp
import pygeon as pg

We create now the grid, since we will use a Raviart-Thomas approximation for ${q}$ we are restricted to simplices. In this example we consider a 2-dimensional grid.

In [2]:
mesh_size = 0.05
# creation of the grid
sd = pg.unit_grid(2, mesh_size, as_mdg=False)
# compute the geometrical properties of the grid
sd.compute_geometry()

With the following code we set the data, in particular the permeability tensor and the scalar source.

In [3]:
key = "flow"

# declare the discretization objects, useful to setup the data
rt0 = pg.RT0(key)
p0 = pg.PwConstants(key)

# set up the data for the flow problem
data = {}

# unitary permeability tensor
inv_perm = pp.SecondOrderTensor(np.ones(sd.num_cells))
parameters = {
    "second_order_tensor": inv_perm,
}
pp.initialize_data(sd, data, key, parameters)

# compute the source term
mass = p0.assemble_mass_matrix(sd)
scalar_source = mass @ p0.interpolate(sd, lambda _: 1)

Once the data are assigned to the grid, we construct the matrices. In particular, the linear system associated with the equation is given as
$$
\left(
\begin{array}{cc} 
A & -B^\top\\
B & 0
\end{array}
\right)
\left(
\begin{array}{c} 
q\\ 
p
\end{array}
\right)
=\left(
\begin{array}{c} 
0\\ 
f
\end{array}
\right)
$$<br>
To construct the saddle-point problem, we rely on the `scipy.sparse` function `block_array`. Once the matrix is created, we also construct the right-hand side containing the source term.

In [ ]:
# construct the local matrices
A = rt0.assemble_mass_matrix(sd, data)
mass_p0 = p0.assemble_mass_matrix(sd, data)
B = mass_p0 @ rt0.assemble_diff_matrix(sd)

# assemble the saddle point problem
spp = sps.block_array(
    [
        [A, -B.T],
        [B, None],
    ],
    format="csc",
)

# get the degrees of freedom for each variable
dof_p, dof_q = B.shape

# assemble the right-hand side
rhs = np.zeros(dof_p + dof_q)
rhs[dof_q:] += scalar_source

We need to solve the linear system, PyGeoN provides a framework for that. Once the problem is solved, we extract the two solutions $q$ and $p$.

In [5]:
# solve the problem
ls = pg.LinearSystem(spp, rhs)
x = ls.solve()

# extract the variables
q = x[:dof_q]
p = x[-dof_p:]

Since the computed $q$ is one value per facet of the grid, for visualization purposes we project the flux in each cell center as vector. We finally export the solution to be visualized by [ParaView](https://www.paraview.org/).

In [6]:
# post process variables
proj_q = rt0.eval_at_cell_centers(sd)
cell_q = (proj_q @ q).reshape((3, -1))
cell_p = p0.eval_at_cell_centers(sd) @ p

save = pp.Exporter(sd, "sol", folder_name="ex1")
save.write_vtu([("cell_p", cell_p), ("cell_q", cell_q)])

In [7]:
# Consistency check
assert np.isclose(np.linalg.norm(cell_p), 1.2588784358524556)
assert np.isclose(np.linalg.norm(cell_q), 5.768072449224256)